In [1]:
import sys
sys.path.append("../h2o-py/build/main")
import h2o
import time
import os
import statistics
import random

versionFromGradle='3.37.0',projectVersion='3.37.0.99999',branch='valenad-PUBDEV-8470-checkpointing',lastCommitHash='271a91cd10b36e841f7e96cbe26ff6155993250b',gitDescribe='jenkins-master-5866-20-g271a91cd10',compiledOn='2022-06-27 01:55:35',compiledBy='adam'


In [2]:
def get_folder_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size


def to_MB(size_in_bytes):
    return size_in_bytes / 1024**2



def measure_model(ntrees, max_depth, checkpoints_path=None):
    times = []
    file_sizes = []
    
    for seed in range(0,10):
        if checkpoints_path!=None: 
            checkpoints_path_unique = checkpoints_path + "_" + str(ntrees) + "_" + str(max_depth) + "_" + str(seed)
            print("Checkpoints path:", checkpoints_path_unique)
        else:
            checkpoints_path_unique=None
            
        creditcards_gbm = H2OGradientBoostingEstimator(ntrees=ntrees,
                                            max_depth=max_depth,
                                            min_rows=1,
                                            score_tree_interval=ntrees+1,
                                            in_training_checkpoints_dir=checkpoints_path_unique,
                                            seed = seed)
        start = time.time()
        creditcards_gbm.train(x=predictors,
                       y=response,
                       training_frame=train)
        end = time.time()
        training_time = end - start
        print("Took", training_time, "s")
        times.append(training_time)
        model_path = h2o.save_model(creditcards_gbm, "models/")
        file_stats = os.stat(model_path)
        model_size = to_MB(file_stats.st_size)
        file_sizes.append(model_size)
        
    print("Mean training times: ", statistics.mean(times))
    print("Mean model size in MegaBytes is", statistics.mean(file_sizes))

    if checkpoints_path != None:
        checkpoints_folder_size = to_MB(get_folder_size(f"{checkpoints_path_unique}"))
        print("Size of all checkpoints is: ", checkpoints_folder_size, 'MB')
        print("Its ", checkpoints_folder_size / statistics.mean(file_sizes), "times more that size of final model")
        
    return times, file_sizes

In [3]:
from h2o.estimators import H2OGradientBoostingEstimator
h2o.init()

creditcards = h2o.import_file("../bigdata/laptop/creditcardfraud/creditcardfraud.csv")

columns = creditcards.columns
predictors = columns[:len(columns)-1]
response = "Class"

creditcards[response] = creditcards[response].asfactor()

train = creditcards

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_311"; Java(TM) SE Runtime Environment (build 1.8.0_311-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.311-b11, mixed mode)
  Starting server from /Users/adam/h2o-3/build/h2o.jar
  Ice root: /var/folders/2z/9gdhqbns0djdj9crb242sgc00000gn/T/tmp10_iht84
  JVM stdout: /var/folders/2z/9gdhqbns0djdj9crb242sgc00000gn/T/tmp10_iht84/h2o_adam_started_from_python.out
  JVM stderr: /var/folders/2z/9gdhqbns0djdj9crb242sgc00000gn/T/tmp10_iht84/h2o_adam_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.37.0.99999
H2O_cluster_version_age:,4 minutes
H2O_cluster_name:,H2O_from_python_adam_dpmb2h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
slowdowns = []
for ntrees in [100, 200, 500, 1000, 1500, 2000]:
    model = measure_model(ntrees, 30)
    model_checkpoints = measure_model(ntrees, 30, "checkpoints_gbm")
    display("Without checkpoints times", model)
    display("With checkpoints times", model_checkpoints)
    slowdown = statistics.mean(model_checkpoints[0])/statistics.mean(model[0]) - 1
    slowdowns.append(slowdown)
    print("Training time is:", slowdown*100, "% different with checkpoints")
    
display("Slowdowns", slowdowns)
print("Training time slowdown average is:", statistics.mean(slowdowns))

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 33.4317569732666 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 33.27121710777283 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 31.909560203552246 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 31.636754989624023 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 31.72126603126526 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 31.708994150161743 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 34.28730392456055 s
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Took 35.95831298828125 s
gbm Model Build progress: |███████████

'Without checkpoints times'

([33.4317569732666,
  33.27121710777283,
  31.909560203552246,
  31.636754989624023,
  31.72126603126526,
  31.708994150161743,
  34.28730392456055,
  35.95831298828125,
  33.62814712524414,
  33.56273412704468],
 [1.4883661270141602,
  1.4884281158447266,
  1.4884319305419922,
  1.4884357452392578,
  1.4884471893310547,
  1.4884347915649414,
  1.4884347915649414,
  1.4884471893310547,
  1.4884510040283203,
  1.488450050354004])

'With checkpoints times'

([33.843544006347656,
  33.82450771331787,
  34.1566059589386,
  34.20288801193237,
  35.25788402557373,
  34.07809495925903,
  35.543802976608276,
  34.49162411689758,
  33.99303317070007,
  34.301186084747314],
 [1.488499641418457,
  1.4884929656982422,
  1.488485336303711,
  1.4884824752807617,
  1.4884862899780273,
  1.4884700775146484,
  1.4884977340698242,
  1.4884977340698242,
  1.4884700775146484,
  1.488490104675293])

...

([272.817174911499,
  271.63794898986816,
  274.88929200172424,
  277.2193431854248,
  277.44274616241455,
  280.67250895500183,
  281.95521306991577,
  281.5670781135559,
  281.25603103637695,
  282.94080686569214],
 [6.224216461181641,
  6.22420597076416,
  6.224215507507324,
  6.224140167236328,
  6.224234580993652,
  6.224040985107422,
  6.224025726318359,
  6.224079132080078,
  6.224143028259277,
  6.224170684814453])

Training time is: 5.930897115037825 % different with checkpoints


'Slowdowns'

[0.0379840345822009,
 -0.00839777141368403,
 -0.008743217293592842,
 0.017895697672067223,
 0.040524621621725654,
 0.059308971150378254]

Training time slowdown average is: 0.02309538938651586
